Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import random
from matplotlib import pyplot as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  #valid_dataset = train_dataset[:10000]
  #valid_labels = train_labels[:10000]
  #test_dataset = train_dataset[10000:20000]
  #test_labels = train_labels[10000:20000]
  #train_dataset = train_dataset[20000:]
  #train_labels = train_labels[20000:]
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [14]:
#tf.logging.set_verbosity(tf.logging.INFO)

num_steps = 20001
batch_size = 128
validate_every = 500

graph = tf.Graph()

with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size), name='train_dataset')
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels), name='train_labels')
    tf_valid_dataset = tf.constant(valid_dataset, name='valid_dataset')
    tf_valid_labels = tf.constant(valid_labels, name='valid_labels')
    tf_test_dataset = tf.constant(test_dataset, name='test_dataset')
    
    layers = [image_size*image_size, 1024, 512, 256, num_labels]
    weights = []
    biases = []
    for i, v in enumerate(layers[1:]):
        with tf.name_scope('hidden{}'.format(i)) as scope:
            weights.append(tf.Variable(tf.truncated_normal([layers[i], v], stddev=np.sqrt(2.0 / layers[i])), name='W{}'.format(i)))
            biases.append(tf.Variable(tf.zeros([v]), name='B{}'.format(i)))
        
    def forward(data, apply_dropout):
        layer_output = data
        for i, v in enumerate(zip(weights, biases)):
            W = v[0]
            B = v[1]
            layer_output = tf.matmul(layer_output, W) + B
            if not i == len(biases)-1:
                layer_output = tf.nn.relu(layer_output)
                if apply_dropout:
                    layer_output = tf.nn.dropout(layer_output, keep_prob=0.5)
        return layer_output
                
    output = forward(tf_train_dataset, True)
    valid_output = forward(tf_valid_dataset, False)

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=tf_train_labels))
    #tf.summary.scalar('loss', loss)
    #valid_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=valid_output, labels=tf_valid_labels))
    
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.5, global_step, 5000, 0.80, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    train_prediction = tf.nn.softmax(output)
    valid_prediction = tf.nn.softmax(valid_output)
    test_prediction = tf.nn.softmax(forward(tf_test_dataset, False))


config = tf.ConfigProto()
config.log_device_placement=False

import time
t = 0


with tf.Session(graph=graph, config=config) as session:
    train_writer = tf.summary.FileWriter( './logs/my/', session.graph)
    tf.global_variables_initializer().run()
    for step in range(num_steps):
        t1 = time.time()

        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        #merge = tf.summary.merge_all()

        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        #train_writer.add_summary(summary, step)
        
        if (step % validate_every == 0):
            #loss_graph.append(step, {'valid_loss': valid_loss.eval(), 'training_loss': l})
            #training_acc = accuracy(predictions, batch_labels)
            valid_acc = accuracy(valid_prediction.eval(), valid_labels)
            #acc_graph.append(step, {'training_acc': training_acc, 'valid_acc': valid_acc})
            print('{:.4}% [{}] '.format(step/num_steps*100, valid_acc), end='')
            #else:
            #    #loss_graph.append(step, {'training_loss': l})
            print('avg delta: {}'.format(t))
        t = t*0.9 + (time.time()-t1)*0.1
        
    print('\n\nTest accuracy: {}'.format(accuracy(test_prediction.eval(), test_labels)))

0.0% [22.35] avg delta: 0
2.5% [84.34] avg delta: 0.003497722457036067
5.0% [85.51] avg delta: 0.0035866979162756667
7.5% [85.66] avg delta: 0.0032766996936728094
10.0% [86.49] avg delta: 0.003262576534860819
12.5% [86.76] avg delta: 0.003610244341891729
15.0% [86.8] avg delta: 0.0035548088061153876
17.5% [87.26] avg delta: 0.0032529709013829544
20.0% [87.65] avg delta: 0.0032296632550648918
22.5% [87.61] avg delta: 0.0037196049200694534
25.0% [88.06] avg delta: 0.003270577170334405
27.5% [88.46] avg delta: 0.003258986908343045
30.0% [88.4] avg delta: 0.003280687450706787
32.5% [88.63] avg delta: 0.0036498075923819015
35.0% [88.71] avg delta: 0.003559644153725296
37.5% [88.85] avg delta: 0.0033433024506445293
40.0% [89.02] avg delta: 0.003270869160219635
42.5% [88.94] avg delta: 0.0033434017006911277
45.0% [89.06] avg delta: 0.003250658720752179
47.5% [88.93] avg delta: 0.003258149460946778
50.0% [89.07] avg delta: 0.0033449763987309514
52.5% [89.37] avg delta: 0.0032632360285814446
55

In [ ]:
print(len(loss_hist))
f, axarr = plt.subplots(3, sharex=True)
axarr[0].plot(loss_hist[8:])
axarr[0].set_title('Loss')
axarr[1].scatter([i*validate_every for i in range(len(training_acc))], training_acc)
axarr[1].set_title('Training accuracy')
axarr[2].scatter([i*validate_every for i in range(len(valid_acc))], valid_acc)
axarr[2].set_title('Validation accuracy')
f.subplots_adjust(left=0, right=2, top=2)
plt.show()

In [ ]:
plt.plot(dropout_values, dropout_acc)
plt.grid(True)
plt.title('Test accuracy')
plt.show()

In [13]:
batch_size = 128
num_hidden_nodes1 = 1024
num_hidden_nodes2 = 512
num_hidden_nodes3 = 256
keep_prob = 0.5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size), name='train_dataset')
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels), name='train_labels')
  tf_valid_dataset = tf.constant(valid_dataset, name='valid_dataset')
  tf_test_dataset = tf.constant(test_dataset, name='test_dataset')
  global_step = tf.Variable(0, name='global_step')

  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal(
        [image_size * image_size, num_hidden_nodes1],
        stddev=np.sqrt(2.0 / (image_size * image_size)))
    , name='W0')
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes1]), name='B0')
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes1, num_hidden_nodes2], stddev=np.sqrt(2.0 / num_hidden_nodes1)), name='W1')
  biases2 = tf.Variable(tf.zeros([num_hidden_nodes2]), name='B1')
  weights3 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes2, num_hidden_nodes3], stddev=np.sqrt(2.0 / num_hidden_nodes2)), name='W2')
  biases3 = tf.Variable(tf.zeros([num_hidden_nodes3]), name='B2')
  weights4 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes3, num_labels], stddev=np.sqrt(2.0 / num_hidden_nodes3)), name='W3')
  biases4 = tf.Variable(tf.zeros([num_labels]), name='B3')
  
  # Training computation.
  lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  drop1 = tf.nn.dropout(lay1_train, 0.5)
  lay2_train = tf.nn.relu(tf.matmul(drop1, weights2) + biases2)
  drop2 = tf.nn.dropout(lay2_train, 0.5)
  lay3_train = tf.nn.relu(tf.matmul(drop2, weights3) + biases3)
  drop3 = tf.nn.dropout(lay3_train, 0.5)
  logits = tf.matmul(drop3, weights4) + biases4
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 5000, 0.80, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  lay2_valid = tf.nn.relu(tf.matmul(lay1_valid, weights2) + biases2)
  lay3_valid = tf.nn.relu(tf.matmul(lay2_valid, weights3) + biases3)
  valid_prediction = tf.nn.softmax(tf.matmul(lay3_valid, weights4) + biases4)
  lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  lay2_test = tf.nn.relu(tf.matmul(lay1_test, weights2) + biases2)
  lay3_test = tf.nn.relu(tf.matmul(lay2_test, weights3) + biases3)
  test_prediction = tf.nn.softmax(tf.matmul(lay3_test, weights4) + biases4)

num_steps = 20001
t = 0
import time

with tf.Session(graph=graph) as session:
  train_writer = tf.summary.FileWriter( './logs/better/', session.graph)
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    t1 = time.time()
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    merge = tf.summary.merge_all()
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    ###train_writer.add_summary(summary, step)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print(t)
    t = t*0.9 + (time.time()-t1)*0.1
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.911421
Minibatch accuracy: 7.8%
Validation accuracy: 16.6%
0
Minibatch loss at step 500: 0.703322
Minibatch accuracy: 82.8%
Validation accuracy: 84.2%
0.003274699597300136
Minibatch loss at step 1000: 0.442894
Minibatch accuracy: 86.7%
Validation accuracy: 85.3%
0.003306173833797725
Minibatch loss at step 1500: 0.619211
Minibatch accuracy: 83.6%
Validation accuracy: 85.8%
0.0034964853491031966
Minibatch loss at step 2000: 0.483866
Minibatch accuracy: 84.4%
Validation accuracy: 86.4%
0.003262906191415224
Minibatch loss at step 2500: 0.576772
Minibatch accuracy: 85.9%
Validation accuracy: 86.8%
0.003306480234204133
Minibatch loss at step 3000: 0.473927
Minibatch accuracy: 83.6%
Validation accuracy: 86.7%
0.005078921399268737
Minibatch loss at step 3500: 0.510051
Minibatch accuracy: 86.7%
Validation accuracy: 87.1%
0.00327543796822104
Minibatch loss at step 4000: 0.452696
Minibatch accuracy: 87.5%
Validation accuracy: 87.8%
0.0033686787783198304
Min

KeyboardInterrupt: 